In [18]:
import pandas as pd
import glob
import json

In [19]:
folders = glob.glob("Data/*")

In [20]:
dictlist = []

for folder in folders:
    if folder == "Data/user":
        continue
    files = glob.glob(folder + "/tweets/*")
    for file in files:
        json_string = open(file,'r',encoding="utf8").read()
        json_dict = json.loads(json_string)
        json_dict["search_term"] = folder[5:].capitalize()
        dictlist.append(json_dict)

In [21]:
df = pd.DataFrame(dictlist)
df = df.replace({'\n': ' '}, regex=True) # remove linebreaks in the dataframe
df = df.replace({'\t': ' '}, regex=True) # remove tabs in the dataframe
df = df.replace({'\r': ' '}, regex=True) # remove carriage return in the dataframe

In [22]:
df.sort_values("search_term",ascending = True,inplace=True,na_position="last")

In [23]:
df

,usernameTweet,ID,text,url,nbr_retweet,nbr_favorite,nbr_reply,datetime,is_reply,is_retweet,user_id,search_term,has_media,medias
0,Rohit_Tn1,1287327756217274375,Already oru movie act pannirukkaru sariya po...,/Rohit_Tn1/status/1287327756217274375,3,0,0,2020-07-26 15:33:49,True,False,1263496177007263750,act,NaN,NaN
22,StanKamanguya,1287327845119909890,You will get myriad reasons as to why undertak...,/StanKamanguya/status/1287327845119909890,0,0,1,2020-07-26 15:34:10,False,False,895317002323144704,act,NaN,NaN
23,_GR19,1287327845157679105,# TPG1 ACT @2M786 @oix6y @ovs_0 @f18_er ...,/_GR19/status/1287327845157679105,2,2,1,2020-07-26 15:34:10,False,False,768900791008624640,act,NaN,NaN
24,elfairyLie92,1287327846294118405,bornfreeone_kis Most of our problems are becau...,/elfairyLie92/status/1287327846294118405,0,0,0,2020-07-26 15:34:10,False,False,374272899,act,NaN,NaN
25,myrandobrando,1287327854003331072,I’ll admit I act a little fake on main but m...,/myrandobrando/status/1287327854003331072,0,0,0,2020-07-26 15:34:12,False,False,1267011209632481280,act,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12481,wf_emerson,1287323351481032704,Sun 05:00: Today: Sunny. A mix of sun and clou...,/wf_emerson/status/1287323351481032704,0,0,0,2020-07-26 15:16:18,False,False,1107171470,Thunderstorm,NaN,NaN
12482,wf_portagpraire,1287323359307534336,Sun 05:00: Today: Sunny. A mix of sun and clou...,/wf_portagpraire/status/1287323359307534336,0,0,0,2020-07-26 15:16:20,False,False,202001769,Thunderstorm,NaN,NaN
12483,wf_thompson,1287323365259304960,Sun 05:00: Today: cool. Showers with risk of a...,/wf_thompson/status/1287323365259304960,0,0,0,2020-07-26 15:16:22,False,False,168845177,Thunderstorm,NaN,NaN
12485,eleanorcastile,1287324165075345408,We had a lovely big thunderstorm yesterday a...,/eleanorcastile/status/1287324165075345408,0,1,0,2020-07-26 15:19:32,True,False,20088540,Thunderstorm,NaN,NaN
